# Decision tree
This is a trial on working of the decision tree

In [25]:
%autosave 20
import math 
import pandas as pd

Autosaving every 20 seconds


In [26]:
#Load the dataset weather
df=pd.read_csv("Sample dataset values.txt")
df

,Sr No,Sensor,Name,Variable,Units,Timestamp,Value,Flagged as Suspect Reading
0,1586958692652188245,sensor03,PER_AIRMON_MESH1976150,PM10,ugm -3,2018-12-13 03:16:00,16.12200,False
1,1586958692652348737,sensor01,PER_AIRMON_MESH1976150,PM 4,ugm -3,2018-12-14 12:46:00,5.19000,False
2,1586958692652681524,sensor02,PER_AIRMON_MESH1976150,NO,ugm -3,2018-12-14 00:30:00,0.08875,False
3,1586958692653617146,sensor01,PER_AIRMON_MESH1976150,PM2.5,ugm -3,2018-12-14 12:46:00,13.84500,False
4,1586958692655668348,sensor02,PER_AIRMON_MESH1976150,PM 4,ugm -3,2018-12-14 00:30:00,1.76000,False
...,...,...,...,...,...,...,...,...
262,1586958692890648280,sensor02,PER_AIRMON_MESH1976150,PM10,ugm -3,2018-12-14 00:42:00,7.38800,False
263,1586958692890824556,sensor01,PER_AIRMON_MESH1976150,PM 4,ugm -3,2018-12-14 13:03:00,4.21000,False
264,1586958692892191120,sensor03,PER_AIRMON_MESH1976150,O3,ppb,2018-12-13 03:20:00,23.01300,False
265,1586958692892201991,sensor01,PER_AIRMON_MESH1976150,NO,ugm -3,2018-12-14 13:04:00,5.38375,False


In [27]:
def uni(col):
    print("There are ",len(df[col].unique())," unique elements  of column ", col," and they are ", df[col].unique())

In [28]:
list(map(uni,df[['Sensor', 'Name', 'Variable', 'Units','Flagged as Suspect Reading']]))

There are  3  unique elements  of column  Sensor  and they are  ['sensor03' 'sensor01' 'sensor02']
There are  1  unique elements  of column  Name  and they are  ['PER_AIRMON_MESH1976150']
There are  8  unique elements  of column  Variable  and they are  ['PM10' 'PM 4' 'NO' 'PM2.5' 'PM1' 'Particle Count' 'O3' 'NO2']
There are  3  unique elements  of column  Units  and they are  ['ugm -3' 'Kgm -3' 'ppb']
There are  2  unique elements  of column  Flagged as Suspect Reading  and they are  [False  True]


[None, None, None, None, None]

In [29]:
df.loc[df['Sensor']=='sensor01','Sensor']=0
df.loc[df['Sensor']=='sensor02','Sensor']=1
df.loc[df['Sensor']=='sensor03','Sensor']=2

In [30]:
df.loc[df['Variable']=='PM10','Variable']=0
df.loc[df['Variable']=='PM 4','Variable']=1
df.loc[df['Variable']=='NO','Variable']=2
df.loc[df['Variable']=='PM2.5','Variable']=3
df.loc[df['Variable']=='PM1','Variable']=4
df.loc[df['Variable']=='Particle Count','Variable']=5
df.loc[df['Variable']=='O3','Variable']=6
df.loc[df['Variable']=='NO2','Variable']=7

In [31]:
df.loc[df['Name']=='PER_AIRMON_MESH1976150','Name']=0

In [32]:
df.loc[df['Units']=='ugm -3','Units']=0
df.loc[df['Units']=='Kgm -3','Units']=1
df.loc[df['Units']=='ppb','Units']=2

In [33]:
df.loc[df['Flagged as Suspect Reading']==False,'Flagged as Suspect Reading']=0
df.loc[df['Flagged as Suspect Reading']==True,'Flagged as Suspect Reading']=1

In [34]:
P,N=df['Flagged as Suspect Reading'].value_counts()
print("correct :",P)
print("incorrect :",N)

correct : 255
incorrect : 12


In [35]:
xtrain=['Sensor', 'Name', 'Variable', 'Units','Value']
ytrain=['Flagged as Suspect Reading']
ytrain
df=df[xtrain+ytrain]
df.head()

['Flagged as Suspect Reading']

,Sensor,Name,Variable,Units,Value,Flagged as Suspect Reading
0,2,0,0,0,16.12200,0
1,0,0,1,0,5.19000,0
2,1,0,2,0,0.08875,0
3,0,0,3,0,13.84500,0
4,1,0,1,0,1.76000,0


In [36]:
IG = pd.DataFrame(columns=['column', 'Info_gain'])

In [37]:
# Function to calculate information gain
def informationGain(p,n):
    if(p==0):
        return 0
    elif(n==0):
        return 0    
    IG= -(p/(p+n))*math.log2(p/(p+n)) - (n/(p+n))*math.log2(n/(p+n))
    return round(IG,3)

In [38]:
IG=informationGain(P,N)
print("Total Information Gain : ",IG)

Total Information Gain :  0.265


In [39]:
Gain = pd.DataFrame(columns=['column', 'gain'])
dependent=''.join(ytrain)

In [40]:
def find_gain(new_df,Gain):
    for col in range(0,len(new_df.columns)-1):
        b=new_df.columns[col]
#         print(b)
        cmn=[]

        cmn=new_df.iloc[:,col].unique()
        ig=[]
        p1=[]
        n1=[]


        for col1 in range(0,len(cmn)):
            p=n=0
#             print(cmn[col1])
            s=new_df.groupby(b).get_group(cmn[col1])
#             print(s)
            p=len(s[s[dependent]==1])
            n=len(s[s[dependent]==0])    
#             print(p)
#             print(n)
            ig.append(informationGain(p,n))
            p1.append(p)
            n1.append(n)
#             print("information gain of ",cmn[col1], "is ",ig[col1])

        E=0.0
        for i in range(0,len(cmn)):
            E=E+(((p1[i]+n1[i])/(P+N))*ig[i])

        print("Entropy of ", b, "is ",round(E,3))
        print("Gain of ",b, "is ",round(IG-E,3))
        Gain=Gain.append([{'column':b,'gain':round(IG-E,3)}], ignore_index=True)

#         print(Gain)
        print("***********")  
    
    return Gain


In [41]:
Gain=find_gain(df,Gain)
print(Gain.sort_values(by=['gain'],ascending=False))

Entropy of  Sensor is  0.227
Gain of  Sensor is  0.038
***********
Entropy of  Name is  0.265
Gain of  Name is  0.0
***********
Entropy of  Variable is  0.124
Gain of  Variable is  0.141
***********
Entropy of  Units is  0.245
Gain of  Units is  0.02
***********
Entropy of  Value is  0.0
Gain of  Value is  0.265
***********
     column   gain
4     Value  0.265
2  Variable  0.141
0    Sensor  0.038
3     Units  0.020
1      Name  0.000


In [42]:
print(Gain.sort_values(by=['gain'],ascending=False))
c=Gain.sort_values(by=['gain'],ascending=False).column.tolist()
c.append(dependent)
c

     column   gain
4     Value  0.265
2  Variable  0.141
0    Sensor  0.038
3     Units  0.020
1      Name  0.000


['Value', 'Variable', 'Sensor', 'Units', 'Name', 'Flagged as Suspect Reading']

In [43]:
df=df[c]
df

,Value,Variable,Sensor,Units,Name,Flagged as Suspect Reading
0,16.12200,0,2,0,0,0
1,5.19000,1,0,0,0,0
2,0.08875,2,1,0,0,0
3,13.84500,3,0,0,0,0
4,1.76000,1,1,0,0,0
...,...,...,...,...,...,...
262,7.38800,0,1,0,0,0
263,4.21000,1,0,0,0,0
264,23.01300,6,2,2,0,0
265,5.38375,2,0,0,0,0
